## CatBoostClassifier

In [8]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from catboost import CatBoostClassifier
#IMPORT AND ENCODE DATA
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
df_train = pd.read_csv('train.csv',index_col=0)
df_test = pd.read_csv('test.csv', index_col=0)
#df_train.reset_index(drop=True)
#df_test.reset_index(drop=True)
# concatenating df1 and df2 along rows

#df_train = pd.concat([df_train, df_test], axis=0)

In [24]:
df_train.head()

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2
1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1
2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1
3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1
4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1


In [25]:
df_train.label.value_counts()

1    37195
0    17995
3    10727
2     8567
7     5042
4      357
6      152
5      141
Name: label, dtype: int64

In [26]:
df_train.org.mode()[0]

'iiitd'

In [27]:

df_train['org'].fillna(df_train['org'].mode()[0], inplace = True)
df_train['tld'].fillna(df_train['tld'].mode()[0], inplace = True)
df_train['mail_type'].fillna(df_train['mail_type'].mode()[0], inplace = True)
df_train['urls'].fillna(df_train['urls'].mean(), inplace=True)
df_train['images'].fillna(df_train['images'].mean(), inplace=True)
df_train['chars_in_subject'].fillna(df_train['chars_in_subject'].mean(), inplace=True)
df_train['chars_in_body'].fillna(df_train['chars_in_body'].mean(), inplace=True)

df_test['org'].fillna(df_test['org'].mode()[0], inplace = True)
df_test['tld'].fillna(df_test['tld'].mode()[0], inplace = True)
df_test['mail_type'].fillna(df_test['mail_type'].mode()[0], inplace = True)
df_test['urls'].fillna(df_test['urls'].mean(), inplace=True)
df_test['images'].fillna(df_test['images'].mean(), inplace=True)
df_test['chars_in_subject'].fillna(df_test['chars_in_subject'].mean(), inplace=True)
df_test['chars_in_body'].fillna(df_test['chars_in_body'].mean(), inplace=True)


df_train['mail_type'] = df_train['mail_type'].str.lower()
df_test['mail_type'] = df_test['mail_type'].str.lower()

In [14]:
df_train

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2
1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1
2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1
3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1
4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80171,"Tue, 30 Jun 2020 05:55:20 -0700",github,com,1,0,multipart/alternative,0,8,0,0,91.0,1307,3
80172,"Fri, 1 May 2015 11:48:55 +0530 (IST)",iiitd,com,0,0,text/html,0,2,1,0,73.0,13464,0
80173,"Tue, 26 May 2020 08:18:43 -0700",google,com,0,0,multipart/alternative,27,85,0,0,46.0,28485,0
80174,"Fri, 14 Dec 2018 09:01:13 +0000 (UTC)",medium,com,0,0,multipart/alternative,32,239,1,1,169.0,105276,0


In [28]:
df_train

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
0,"Mon, 6 Nov 2017 11:13:45 +0100",reply,ebay.in,0,0,multipart/alternative,35,120,0,0,49.0,80027,2
1,"Wed, 14 Feb 2018 11:00:16 -0000",edm,efinmail.com,0,0,multipart/alternative,1,7,0,0,107.0,2961,1
2,"Wed, 6 Jul 2016 19:53:37 +0000",usebackpack,com,0,0,text/html,4,17,0,0,35.0,25149,1
3,"Fri, 11 Oct 2019 11:25:40 +0200",granular,ai,0,0,multipart/mixed,0,0,0,0,15.0,635296,1
4,"Tue, 07 Nov 2017 11:07:18 +0000 (UTC)",github,com,1,0,multipart/alternative,2,11,0,0,49.0,2355,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80171,"Tue, 30 Jun 2020 05:55:20 -0700",github,com,1,0,multipart/alternative,0,8,0,0,91.0,1307,3
80172,"Fri, 1 May 2015 11:48:55 +0530 (IST)",iiitd,com,0,0,text/html,0,2,1,0,73.0,13464,0
80173,"Tue, 26 May 2020 08:18:43 -0700",google,com,0,0,multipart/alternative,27,85,0,0,46.0,28485,0
80174,"Fri, 14 Dec 2018 09:01:13 +0000 (UTC)",medium,com,0,0,multipart/alternative,32,239,1,1,169.0,105276,0


In [29]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80176 entries, 0 to 80175
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              80176 non-null  object 
 1   org               80176 non-null  object 
 2   tld               80176 non-null  object 
 3   ccs               80176 non-null  int64  
 4   bcced             80176 non-null  int64  
 5   mail_type         80176 non-null  object 
 6   images            80176 non-null  int64  
 7   urls              80176 non-null  int64  
 8   salutations       80176 non-null  int64  
 9   designation       80176 non-null  int64  
 10  chars_in_subject  80176 non-null  float64
 11  chars_in_body     80176 non-null  int64  
 12  label             80176 non-null  int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 8.6+ MB


In [30]:
df_test.dtypes

date                 object
org                  object
tld                  object
ccs                   int64
bcced                 int64
mail_type            object
images                int64
urls                  int64
salutations           int64
designation           int64
chars_in_subject    float64
chars_in_body         int64
dtype: object

In [31]:
y = df_train['label'].values
#X = df_train.drop(['label','timezone'],axis=1)
X = df_train.drop(['label'],axis=1)
X["chars_in_subject"] = X["chars_in_subject"].astype(np.int) 

X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80176 entries, 0 to 80175
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              80176 non-null  object
 1   org               80176 non-null  object
 2   tld               80176 non-null  object
 3   ccs               80176 non-null  int64 
 4   bcced             80176 non-null  int64 
 5   mail_type         80176 non-null  object
 6   images            80176 non-null  int64 
 7   urls              80176 non-null  int64 
 8   salutations       80176 non-null  int64 
 9   designation       80176 non-null  int64 
 10  chars_in_subject  80176 non-null  int64 
 11  chars_in_body     80176 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 8.0+ MB


In [32]:

from catboost import CatBoostClassifier
#IMPORT AND ENCODE DATA
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
df_train = pd.read_csv('train.csv',index_col=0)
df_test = pd.read_csv('test.csv', index_col=0)
df_train['org'].fillna(df_train['org'].mode()[0], inplace = True)
df_train['tld'].fillna(df_train['tld'].mode()[0], inplace = True)
df_train['mail_type'].fillna(df_train['mail_type'].mode()[0], inplace = True)
df_train['urls'].fillna(df_train['urls'].mean(), inplace=True)
df_train['images'].fillna(df_train['images'].mean(), inplace=True)
df_train['chars_in_subject'].fillna(df_train['chars_in_subject'].mean(), inplace=True)
df_train['chars_in_body'].fillna(df_train['chars_in_body'].mean(), inplace=True)

df_test['org'].fillna(df_test['org'].mode()[0], inplace = True)
df_test['tld'].fillna(df_test['tld'].mode()[0], inplace = True)
df_test['mail_type'].fillna(df_test['mail_type'].mode()[0], inplace = True)
df_test['urls'].fillna(df_test['urls'].mean(), inplace=True)
df_test['images'].fillna(df_test['images'].mean(), inplace=True)
df_test['chars_in_subject'].fillna(df_test['chars_in_subject'].mean(), inplace=True)
df_test['chars_in_body'].fillna(df_test['chars_in_body'].mean(), inplace=True)


df_train['mail_type'] = df_train['mail_type'].str.lower()
df_test['mail_type'] = df_test['mail_type'].str.lower()
y = df_train['label'].values
#X = df_train.drop(['label','timezone'],axis=1)
X = df_train.drop(['label'],axis=1)
X["chars_in_subject"] = X["chars_in_subject"].astype(np.int) 

X.info()
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=53)

# cat_train=['org','tld','mail_type']
# cat_test =['org','tld','mail_type']


#parameters using the randomized search
from catboost import Pool

clf = CatBoostClassifier(
    iterations=25,
    eval_metric='MultiClass',
    loss_function='MultiClass',
    early_stopping_rounds=10,
    classes_count=8,
    random_strength=2,
    depth=7,
    verbose=3
)

clf.fit(
     X_train, y_train,
    cat_features=[0,1,2,3,4,5,6,7,8,9,10,11], 
    plot=True,
    use_best_model=True,
    eval_set=(X_val, y_val)
)

print(clf.predict(data=X_val))

# X = df
X_test = df_test.drop(['date'],axis=1)
X_test["chars_in_subject"] = X_test["chars_in_subject"].astype(np.int) 




<class 'pandas.core.frame.DataFrame'>
Int64Index: 80176 entries, 0 to 80175
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              80176 non-null  object
 1   org               80176 non-null  object
 2   tld               80176 non-null  object
 3   ccs               80176 non-null  int64 
 4   bcced             80176 non-null  int64 
 5   mail_type         80176 non-null  object
 6   images            80176 non-null  int64 
 7   urls              80176 non-null  int64 
 8   salutations       80176 non-null  int64 
 9   designation       80176 non-null  int64 
 10  chars_in_subject  80176 non-null  int64 
 11  chars_in_body     80176 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 8.0+ MB


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.47962
0:	learn: 1.1914930	test: 1.1819818	best: 1.1819818 (0)	total: 1.56s	remaining: 37.4s
3:	learn: 0.9099869	test: 0.9048844	best: 0.9048844 (3)	total: 5.75s	remaining: 30.2s
6:	learn: 0.8003959	test: 0.7230744	best: 0.7230744 (6)	total: 9.87s	remaining: 25.4s
9:	learn: 0.7846559	test: 0.7058007	best: 0.7058007 (9)	total: 13.2s	remaining: 19.8s
12:	learn: 0.7614319	test: 0.6790260	best: 0.6790260 (12)	total: 17.3s	remaining: 15.9s
15:	learn: 0.7510400	test: 0.6678748	best: 0.6678748 (15)	total: 20.8s	remaining: 11.7s
18:	learn: 0.7404311	test: 0.6561557	best: 0.6561557 (18)	total: 24.8s	remaining: 7.84s
21:	learn: 0.7345066	test: 0.6505680	best: 0.6505680 (21)	total: 28.7s	remaining: 3.92s
24:	learn: 0.7162732	test: 0.6363635	best: 0.6363635 (24)	total: 32.8s	remaining: 0us

bestTest = 0.6363635071
bestIteration = 24

[[3]
 [1]
 [1]
 ...
 [0]
 [1]
 [0]]


## XGBoostClassifier 

In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Read csvs
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

train_df=train_df.assign(mail_type=train_df['mail_type'].str.split('/')).explode('mail_type')
test_df=test_df.assign(mail_type=test_df['mail_type'].str.split('/').explode('mail_type'))

#imputing missing values (mode/mean)
train_df.reset_index(drop=True)
test_df.reset_index(drop=True)


train_df['org'].fillna(train_df['org'].mode()[0], inplace = True)
train_df['tld'].fillna(train_df['tld'].mode()[0], inplace = True)
train_df['mail_type'].fillna(train_df['mail_type'].mode()[0], inplace = True)
train_df['urls'].fillna(train_df['urls'].mean(), inplace=True)
train_df['images'].fillna(train_df['images'].mean(), inplace=True)
train_df['chars_in_subject'].fillna(train_df['chars_in_subject'].mean(), inplace=True)
train_df['chars_in_body'].fillna(train_df['chars_in_body'].mean(), inplace=True)

test_df['org'].fillna(test_df['org'].mode()[0], inplace = True)
test_df['tld'].fillna(test_df['tld'].mode()[0], inplace = True)
test_df['mail_type'].fillna(test_df['mail_type'].mode()[0], inplace = True)
test_df['urls'].fillna(test_df['urls'].mean(), inplace=True)
test_df['images'].fillna(test_df['images'].mean(), inplace=True)
test_df['chars_in_subject'].fillna(test_df['chars_in_subject'].mean(), inplace=True)
test_df['chars_in_body'].fillna(test_df['chars_in_body'].mean(), inplace=True)


train_df['mail_type'] = train_df['mail_type'].str.lower()
test_df['mail_type'] = test_df['mail_type'].str.lower()
#cleaning training dataset

train_df['date'] = train_df['date'].str.replace(r' \(.*\)', '')
train_df['date'] = train_df['date'].str.rstrip()
train_df['date'] = train_df['date'].str.replace('-0000', '+0000')
#some data points have 0580 as the time which can be changed to 0530 since 0530 appears the mmost
train_df['date'] = train_df['date'].str.replace('0580', '0530')
!pip3 install dateparser
import dateparser
#using dateparser since different formats of dates are present. this takes time, but better than manually cconverting each format to unified one
train_df['date'] = train_df['date'].apply(dateparser.parse, languages=['en'])

#cleaning test dataset (same as above)

test_df['date'] = test_df['date'].str.replace(r' \(.*\)', '')
test_df['date'] = test_df['date'].str.rstrip()
test_df['date'] = test_df['date'].str.replace('-0000', '+0000')
test_df['date'] = test_df['date'].str.replace('0580', '0530')
test_df['date'] = test_df['date'].apply(dateparser.parse, languages=['en'])
train_df['year'] = train_df['date'].apply(lambda x:x.year)
train_df['month'] = train_df['date'].apply(lambda x:x.month)
train_df['day'] = train_df['date'].apply(lambda x:x.day)
train_df['hour'] = train_df['date'].apply(lambda x:x.hour)
train_df['minute'] = train_df['date'].apply(lambda x:x.minute)
train_df['second'] = train_df['date'].apply(lambda x:x.second)
train_df['weekday'] = train_df['date'].apply(lambda x:x.weekday())


test_df['year'] = test_df['date'].apply(lambda x:x.year)
test_df['month'] = test_df['date'].apply(lambda x:x.month)
test_df['day'] = test_df['date'].apply(lambda x:x.day)
test_df['hour'] = test_df['date'].apply(lambda x:x.hour)
test_df['minute'] = test_df['date'].apply(lambda x:x.minute)
test_df['second'] = test_df['date'].apply(lambda x:x.second)
test_df['weekday'] = test_df['date'].apply(lambda x:x.weekday())

# #using dateparser since different formats of dates are present. this takes time, but better than manually cconverting each format to unified one
# train_df['date'] = train_df['date'].apply(dateparser.parse, languages=['en'])

# #Rescaling features age, trestbps, chol, thalach, oldpeak.
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# features = [['ccs', 'bcced', 'images', 'urls', 'salutations','designation','chars_in_subject','chars_in_body','label']]
# for feature in features:
#     train_df[feature] = scaler.fit_transform(train_df[feature])
#     train_df=train_df[feature]
#     #test_df[feature] = scaler.fit_transform(test_df[feature])

# features = [['ccs', 'bcced', 'images', 'urls', 'salutations','designation','chars_in_subject','chars_in_body']]
# for feature in features:
#     test_df[feature] = scaler.fit_transform(test_df[feature])
#     test_df=test_df[feature]

#try
!pip install category_encoders
from category_encoders import OrdinalEncoder
mapping = [{'col': 'label', 'mapping': {"Updates": 0,  "Personal": 1, "Promotional": 2, "Forums":3, "Purchases:":4, "Travel":5, "Spam":6, "Social":7}}]
OrdinalEncoder(cols=['label'], mapping=mapping).fit(train_df).transform(train_df)
#OrdinalEncoder(cols=['label'], mapping=mapping).fit(test_df).transform(test_df)
from category_encoders import OneHotEncoder
OneHotEncoder(cols=['mail_type','ccs', 'bcced', 'images', 'urls', 'salutations','designation']).fit(train_df).transform(train_df)

## Filtering column "mail_type"
train_x = train_df[['ccs', 'bcced', 'images', 'urls', 'salutations','designation','chars_in_subject','chars_in_body']]
#train_x=train_df
train_x = train_x.fillna(value='None')
train_y = train_df[['label']]

test_x = test_df[['ccs', 'bcced', 'images', 'urls', 'salutations','designation','chars_in_subject','chars_in_body']]
#test_x=train_df
test_x = test_x.fillna(value='None')


# ## Do one hot encoding of categorical feature
# feat_enc = OneHotEncoder()
# feat_enc.fit(np.vstack([train_x, test_x]))
# train_x_featurized = feat_enc.transform(train_x)
# test_x_featurized = feat_enc.transform(test_x)

# from numpy import loadtxt
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split
# #fit model no training data
# model = XGBClassifier()
# model.fit(train_x, train_y)
# print(model)

import xgboost as xgb
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
 
# boston = load_boston()
x, y = train_x,train_y
dmatrix = xgb.DMatrix(data=x, label=y)
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 8,
    'gamma': 0.1,
    'max_depth': 3,
    'lambda': 2,
    'subsample': 0.7,
    'seed': 1000,
    'nthread': 4,
    'learning_rate':1.05,
    'random_state':20,
    'eta': 0.01, 
    'colsample_bytree': 0.7,
    'min_child_weight': 3,}


# params={ 'objective':'reg:squarederror',
#          'max_depth': 6, 
#          'colsample_bylevel':0.01,
#          'learning_rate':0.9,
#          'random_state':20}
# cv_results = xgb.cv(dtrain=dmatrix, params=params, nfold=10, metrics={'rmse'}, as_pandas=True, seed=20, num_boost_round=1000)

# print('RMSE: %.2f' % cv_results['test-rmse-mean'].min())

dtrain = xgb.DMatrix(train_x, train_y)
dtest = xgb.DMatrix(test_x)
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)
y_pred = model.predict(dtest)
# pred = model.predict(dtest)

# params = {
#     'booster': 'gbtree',
#     'objective': 'multi:softmax',
#     'num_class': 8,
#     'gamma': 0.1,
#     'max_depth': 10,
#     'lambda': 2,
#     'subsample': 0.7,
    # 'colsample_bytree': 0.7,
    # 'min_child_weight': 3,
    # 'silent': 0,
    # 'eta': 0.01,
    # 'seed': 1000,
    # 'nthread': 4,
# }
# dtrain = xgb.DMatrix(X_training, y_training)
# dtest = xgb.DMatrix(X_test_data, y_test_data)
# num_rounds = 50
# model = xgb.train(params, dtrain, num_rounds)
# pred = model.predict(dtest)

# # make predictions for test data
# y_pred = model.predict(test_x)
y_pred = [round(value) for value in y_pred]

#accuracy = accuracy_score('/content/sample_submission.csv', y_pred)
#print("The accuracy of prediction is: ", accuracy)

# ## Save results to submission file
pred_df = pd.DataFrame(y_pred, columns=['label'])
pred_df.to_csv("xgb.csv", index=True, index_label='Id')

<ipython-input-1-909b659f7e81>:42: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['date'] = train_df['date'].str.replace(r' \(.*\)', '')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 292 kB 4.9 MB/s 


<ipython-input-1-909b659f7e81>:54: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df['date'] = test_df['date'].str.replace(r' \(.*\)', '')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 691 kB/s 


## Random Forest classifier


In [3]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Read csvs
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

train_df=train_df.assign(mail_type=train_df['mail_type'].str.split('/')).explode('mail_type')
test_df=test_df.assign(mail_type=test_df['mail_type'].str.split('/').explode('mail_type'))

#create a text/image ratio
# train_df['ratio']=(train_df['images']+train_df['urls']+train_df['chars_in_body'])/train_df['chars_in_body']
# test_df['ratio']=(test_df['images']+test_df['urls']+test_df['chars_in_body'])/test_df['chars_in_body']



## Filtering column "mail_type"
train_x = train_df[['designation','salutations']]
train_x = train_x.fillna(value='None')
train_y = train_df[['label']]

test_x = test_df[['designation','salutations']]
test_x = test_x.fillna(value='None')

## Do one hot encoding of categorical feature
# feat_enc = OneHotEncoder()
# feat_enc.fit(np.vstack([train_x, test_x]))
# train_x_featurized = feat_enc.transform(train_df['mail_type'])
# test_x_featurized = feat_enc.transform(test_df['mail_type'])

## Train a simple KNN classifier using featurized data
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_x, train_y)
pred_y=clf.predict(test_x)
# =neigh = KNeighborsClassifier(n_neighbors=3)
# neigh.fit(train_x, train_y)
# pred_y = neigh.predict(test_x)

## Save results to submission file
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("knn_RF.csv", index=True, index_label='Id')

<ipython-input-3-d1a7ab34bbb4>:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(train_x, train_y)
